In [ ]:
                                                                                # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
#reading CSV with pandas
Taxonomy = pd.read_csv('../input/birdclef-2022/eBird_Taxonomy_v2021.csv')
test = pd.read_csv('../input/birdclef-2022/test.csv')
train_metadata = pd.read_csv('../input/birdclef-2022/train_metadata.csv')
submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')
from os import path
from pydub import AudioSegment
import seaborn as sns

# files
src = "../input/birdclef-2022/train_audio/afrsil1/XC395771.ogg"#this audio have 4.5 rating
dst = "/kaggle/working/XC395771.wav"

# convert ogg to wav
sound = AudioSegment.from_ogg(src)
sound.export(dst, format="wav")

#transform into dataframe
df_taxonomy = pd.DataFrame(Taxonomy)

df_test = pd.DataFrame(test)

df_train = pd.DataFrame(train_metadata)

df_submission = pd.DataFrame(submission)

df_taxonomy.head()







# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_test.head()

In [ ]:
df_train.size

In [ ]:
df_train['primary_label']

In [ ]:
df_train.columns
df_train.head()

In [ ]:
df_train['filename'][16]

In [ ]:
df_train['rating']
y_axis = df_train['rating']
x_axis = df_train['primary_label']
plt.bar(x_axis, y_axis)
plt.show()

In [ ]:
df_submission.head()

In [ ]:
#opening the scored bird file
with open('../input/birdclef-2022/scored_birds.json','r') as f:
    data = json.load(f)

In [ ]:
print(data)

In [ ]:
sample_rate, samples = wavfile.read('/kaggle/working/XC395771.wav')


In [ ]:
len(samples)

In [ ]:
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

In [ ]:
#Plotting Amplitude of a wav file against time
from scipy.io.wavfile import read
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
input_data = read("/kaggle/working/XC395771.wav")
audio = input_data[1]
plt.plot(audio[0:200000])#6seconds of video
plt.ylabel("Amplitude")
plt.xlabel("Time")
plt.show()

hypothesis: we need to consider the distancies between peaks with aproximately the same amplitude but only the ones which are close to each other, for example, the first two peaks that can be seeing in the image above. I think one of the features can be the average speed at which the bird repeat the same frequency. It is easier to understand when you listen the first 6 second of the audio and see the image above at the same time.

In [ ]:
#Lets try to look for periodicity in this audio

from statsmodels.graphics.tsaplots import plot_acf
plot_acf(audio)
plt.show()

In [ ]:
#Creating an spectrogram of the same file
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
plt.rcParams["figure.figsize"] = [7.00, 3.50]
plt.rcParams["figure.autolayout"] = True
sample_rate, samples = wavfile.read('/kaggle/working/XC395771.wav')
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
plt.pcolormesh(times, frequencies, 10 * np.log10(spectrogram),shading = 'auto')
plt.figure(figsize = (3,3))

plt.show

In [ ]:
#We calculate the discrete Fourier Transformation for our signal

fft_spectrum = np.fft.rfft(input_data[1])

In [ ]:
fft_spectrum

In [ ]:
#range of frequencies
freq = np.fft.rfftfreq(input_data[1].size, d = 1./sample_rate)

In [ ]:
fft_spectrum_abs = np.abs(fft_spectrum)

In [ ]:
# we plot FFT
plt.plot(freq,fft_spectrum_abs)
plt.xlabel("frequency, Hz")
plt.ylabel("Amplitude, units")
plt.show()

Hypothesis:  We may fit a gaussian here and get sigma values. Depending on the bird we may fit more than one gaussian and get more than one sigma value.

In [ ]:
np.shape(spectrogram)

In [ ]:
len(frequencies)

In [ ]:
len(times)

In [ ]:
plt.figure(figsize=(20, 6))

sns.countplot(df_train['rating'])
plt.title("Distribution of Ratings", fontsize=20)

plt.show()

In [ ]:
plt.figure(figsize=(20, 6))

sns.countplot(df_train['primary_label'])
plt.xticks(rotation=90)
plt.title("Distribution of Primary Labels", fontsize=20)

plt.show

In [ ]:
plt.figure(figsize=(20, 6))

sns.countplot(df_train['scientific_name'])
plt.xticks(rotation=90)
plt.title("Distribution of scientific names", fontsize=20)

plt.show()

In [ ]:
plt.figure(figsize=(20, 6))

sns.countplot(df_train['type'][3000:4000])
plt.xticks(rotation=90)
plt.title("type", fontsize=20)

plt.show()

In [ ]:
plt.figure(figsize=(20, 6))

sns.countplot(df_train['type'])
plt.title("Distribution of Primary types", fontsize=30)
plt.tick_params(axis = 'x', which = 'both', bottom = False,top = False, labelbottom = False )
plt.show()

In [ ]:
#Elimino los elementos con un rating menor que 2.5, 
#lo que elimina tb los elementos que no tienen rating
#for i in df_train.index:
    #if df_train['rating'][i]<2.5:
       # df_train.drop([i],axis = 0, inplace = True)


In [ ]:
#Lets check the rating of the scored birds

index_row_scored = []
for i in data:
    a=i.strip('"')
    for j in range(len(df_train["primary_label"])):
        if df_train["primary_label"][j] == a:
            index_row_scored.append(j)
label_row_scored = []
for i in index_row_scored:
    label_row_scored.append(df_train["rating"][i])
    

plt.figure(figsize=(20, 6))
sns.countplot(label_row_scored)
plt.title("rating of scored birds", fontsize=20)

plt.show()

In [ ]:
#let's check the primary_label of scored birds

label_row_scored = []
for i in index_row_scored:
    label_row_scored.append(df_train["primary_label"][i])
    

plt.figure(figsize=(20, 6))
sns.countplot(label_row_scored)
plt.title("primary_label scored birds", fontsize=20)

plt.show()

List with the rated audios:

In [ ]:

from os import listdir
from os.path import isfile, join
rated_list = []
for i in data:
    onlyfiles = [f for f in listdir("../input/birdclef-2022/train_audio/"+i) if isfile(join("../input/birdclef-2022/train_audio/"+i,f))]
    rated_list.append(i)
    rated_list.append(onlyfiles)
    

In [ ]:
#Distribution of time of each scored audio
#Times goes from 0 to max(time_row_scored)=1414 sec.
counter = 0
time_row_scored = []
for i in index_row_scored:
    counter = counter +1
    if df_train["time"][i][0] not in ["?","x"] and df_train["time"][i][1] not in [":"]:
        time = int(df_train["time"][i][0])*10*60 + int(df_train["time"][i][1])*60 + int(df_train["time"][i][3])*10 +int(df_train["time"][i][4])
        time_row_scored.append(time)
    elif df_train["time"][i][0] not in ["?","x"]:
        time = int(df_train["time"][i][0])*60 + int(df_train["time"][i][2])*10 +int(df_train["time"][i][3])
        time_row_scored.append(time)
        
    
plt.figure(figsize=(20, 6))
sns.countplot(time_row_scored)
plt.title("time scored birds, max = 1414 sec.", fontsize=20)
plt.tick_params(axis = 'x', which = 'both', bottom = False,top = False, labelbottom = False )
plt.xticks(rotation=90)

plt.show()

In [ ]:
#Let's calculate how many seconds of audio we have for each scored bird if possible and 
#let save it in a dictionary

counter = 0
time_row_scored = []
for i in index_row_scored:
    counter = counter +1
    if df_train["time"][i][0] not in ["?","x"] and df_train["time"][i][1] not in [":"]:
        time = int(df_train["time"][i][0])*10*60 + int(df_train["time"][i][1])*60 + int(df_train["time"][i][3])*10 +int(df_train["time"][i][4])
        time_row_scored.append(time)
    elif df_train["time"][i][0] not in ["?","x"]:
        time = int(df_train["time"][i][0])*60 + int(df_train["time"][i][2])*10 +int(df_train["time"][i][3])
        time_row_scored.append(time)
    else:
        time_row_scored.append(['Undefined'])


    

In [ ]:
from collections import defaultdict
label_row_scored.sort()
outputlist = defaultdict(list)
for A, B in zip(label_row_scored, time_row_scored):
    outputlist[A].append(B)
time_scored_sum = []
for i in data:
    try:
        suma = sum(outputlist[i])
        time_scored_sum.append(suma)
    except:
        time_scored_sum.append(["?"])

In [ ]:
dict_species_time = dict(zip(data,time_scored_sum))

In [ ]:
dict_species_time

In [ ]:
# As there are not so many scored birds, it can be interesting to get spectrograms of 
#high quality audios, and listen those audios while we check the frequencies, intensities, etc
#df_high = df_train
#for i in df_high.index:
    #if df_high['rating'][i]<3.5:
        #df_high.drop([i],axis = 0, inplace = True)

In [ ]:
#for j in data:
   # for i in df_high.index:
        #if df_high['primary_label'][i]==j:
            #print(df_high['filename'][i])
#label_row_scored

In [ ]:
#we can try to find how many audios we have of each scored species and check the rating of
# them, for crehon for example, there are only 2 audios and with low quality.
scoredbird_rating = []
for i in index_row_scored:
    try:
        scoredbird_rating.append(str(df_train['primary_label'][i])+ " " +str(df_train['rating'][i]))
    except:
        print('error')
scoredbird_rating
plt.figure(figsize=(20, 6))
sns.countplot(scoredbird_rating)
plt.xticks(rotation=90)
plt.title("rating of scored birds", fontsize=20)

plt.show()


we can see that for crahon the best rating is 2.5, for puaioh 3.5. there are many species with not so many audios, for example omao, puaioh, crahon...

In [ ]:
# with scorebird_rating.index('bird rating') we can localize the index for the best rating of each bird
scoredbird_rating.index('akiapo 5.0')

In [ ]:
#to find them in the dataset we write the indexes on index_row_scored
index_row_scored[10]

In [ ]:
print(str(df_train['primary_label'][42]) + ' ' + str(df_train['rating'][42]))

In [ ]:
print(str(df_train['filename'][42]))

In [ ]:
#the list with the indexes like line [80] is

indexes = [10,17,30,78,88,97,105,111,138,152,160,163,175,501,525,603,611,625,629,1129,1205]

In [ ]:
#let's get our data from the dataframe
indexes_high_rate = []
for i in indexes:
    indexes_high_rate.append(index_row_scored[i])

print(df_train['primary_label'][indexes_high_rate], df_train['rating'][indexes_high_rate])

    
    

In [ ]:
#Let's loop our first code with the random audio but for our list of rated birds
for i in indexes_high_rate:
    src = "../input/birdclef-2022/train_audio/" + str(df_train['filename'][i])
    dst = "/kaggle/working/" + str(df_train['primary_label'][i]) + ".wav"

# convert ogg to wav
    sound = AudioSegment.from_ogg(src)
    sound.export(dst, format="wav")
    
    plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    input_data = read(dst)
    audio = input_data[1]
    print(audio)
    plt.plot(audio)
    plt.ylabel("Amplitude"+' ' +str(df_train['primary_label'][i]) + ' ' + str(df_train['rating'][i]))
    plt.xlabel("Time")
    plt.show()


In [ ]:
#in some cases we have multiple signal. thats because we have stereo and mono audios.

In [ ]:
#Let's start MFCC for one of the audios and then apply it to everyone in order to get 
#features of all the audios

data_, sample_rate_ = librosa.load('/kaggle/working/XC395771.wav')
mfccs = librosa.feature.mfcc(y=data_, sr=sample_rate, n_mfcc=40)
print(mfccs.shape)
print(mfccs)


In [ ]:
#Let's do the same for all the audios
def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #in order to find out scaled feature we do mean of transpose of value
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features


In [ ]:
#Next code works but I will leave it as comment because it takes like 10 hours to get 
# all the features for each audio.
##from tqdm import tqdm
# Now we iterate through every audio file and extract features 
# using Mel-Frequency Cepstral Coefficients
##extracted_features=[]
##for index_num,row in tqdm(df_train.iterrows()):
    ##file_name = os.path.join(os.path.abspath("../input/birdclef-2022/train_audio/"),str(row["filename"]))
    ##final_class_labels=row["primary_label"]
    ##data=features_extractor(file_name)
    ##extracted_features.append([data,final_class_labels])

In [ ]:
#We will use a dataframe with the scored birds just to see that everything works fine 
#without spending so much time
df_scored = df_train.copy()
for i in df_train.index:
    #if df_train['rating'][i]<2.5:
        #df_train.drop([i],axis = 0, inplace = True)
    if i not in index_row_scored:
        df_scored.drop([i],axis = 0, inplace = True)
df_scored

In [ ]:
#Let's see if we get more accuracy when we drop elements with less than 2.5 rating
#for i in df_scored.index:
   # if df_scored['rating'][i]<2.5:
        #df_scored.drop([i],axis = 0, inplace = True)

In [ ]:
#We will use a method that stract features from the audios
from tqdm import tqdm
# Now we iterate through every audio file and extract features 
# using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(df_scored.iterrows()):
    file_name = os.path.join(os.path.abspath("../input/birdclef-2022/train_audio/"),str(row["filename"]))
    final_class_labels=row["primary_label"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
len(extracted_features)

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','primary_label'])
extracted_features_df.head()

In [ ]:
#Now we split the data into train and test
from imblearn.over_sampling import RandomOverSampler #we improved accuracy with this!
oversample = RandomOverSampler(sampling_strategy='minority')
# Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['primary_label'].tolist())
# Label Encoding -> Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))
### Train Test Split
from sklearn.model_selection import train_test_split
#X,y = oversample.fit_resample(X, y)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
# No of classes
num_labels=y.shape[1]

In [ ]:

#import numpy as np
#import matplotlib.pyplot as plt

#from sklearn import svm, datasets
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import ConfusionMatrixDisplay

#classifier = svm.SVC(kernel="linear", C=0.01).fit(X_train, y_train)

#np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
#titles_options = [
    #("Confusion matrix, without normalization", None),
    #("Normalized confusion matrix", "true"),
#]

#for title, normalize in titles_options:
   # disp = ConfusionMatrixDisplay.from_estimator(
        #classifier,
       # X_test,
        #y_test,
        #cmap=plt.cm.Blues,
        #normalize=normalize,
    #)
    #disp.ax_.set_title(title)

    #print(title)
    #print(disp.confusion_matrix)

#plt.show()

In [ ]:

model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
num_epochs = 200
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
model.predict(X_test)[0]

To fully understand what we are doing and change our code propperly we will make a little explanation

In [ ]:
y = np.array(extracted_features_df['primary_label'].tolist())

In [ ]:
y

we our dataset getting x and y being x the feature column and y the prymary label column

In [ ]:
labelencoder.fit_transform(y)

we trnasform into an array with the  names as numbers

In [ ]:
y = to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

now our y is an array of arrais. In first array we see, that the  first element is 1, that means the first kind (akiapo), the second array is the same because for the next audio we have the same bird(akiapo). so this is refered to the audios and the kind of bird we have

In [ ]:
X = np.array(extracted_features_df['feature'].tolist())

In [ ]:
X

For X we have the list of audios with the features. first string we see is associated to the first audios and the number with commmas are the features of such audio

In [ ]:
#X,y = oversample.fit_resample(X,y)

In [ ]:
X.shape

we have increase the number of samples in order to change accuracy

In [ ]:
X_test.shape
X_train.shape
X.shape

We got 20 per cent of the samples for X_test and 80 per cent for X_train . the same for y

In [ ]:
y

In [ ]:
y_test.shape

In [ ]:

# Let's check the accuracy!
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
model.conf = 0.25

In [ ]:
predict_x = model.predict(X_test)
classes_x = np.argmax(predict_x, axis = 1)
predict_x[1]

We have improve from the accuracy from 0.43 to 0.58 just using RandomOverSampler in categorical cross entropy loss :D. I tried others like SMOTE but I got errors because there is a class with just 2 samples and n_samples must be > n_labels with those algorithms. We get accuracy of 0.53 with poisson loss. 0.54 for KLDivergence loss. 0.56 for binary cross entropy loss. 0.65 for MeanSquaredError. 0.66 for MeanAbsoluteError. 0.65 for MeanAbsolutePercentageError. 0.62 for MeanSquaredLogarithmicError. 0.65 for huber. 0.65 for LogCosh. 0.66 for Hinge. 0.66 for SquaredHinge. 0.63 for CategoricalHinge.
We have improved accuracy again :D. 

It looks like the accuracy depends a lot with tne random oversample, so each time we train the neural network we will get different accuracy for the same loss. It is important to find the best oversample and save it.

In [ ]:
#Let's see how to submit our results
#filename = 
#audio, sample_rate = librosa.load(filename, res_type = 'kaiser_fast')
#mfcc_features = librosa.load(y = audio, sr = sample_rate,n_mfcc = 40)
#mfccs_scaled_features = np.mean(mfccs_features.T,axis = 0)

#mfcc_scaled_features = mfcc

In [ ]:
test_audio_dir = '../input/birdclef-2022/test_soundscapes/'
file_list = [f.split('.')[0] for f in sorted(os.listdir(test_audio_dir))]

In [ ]:
print('Number of test soundscapes:', len(file_list))

In [ ]:
with open('../input/birdclef-2022/scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)

In [ ]:
pred = {'row_id': [], 'target': []}

In [ ]:
# Process audio files and make predictions
for afile in file_list:
    
    # Complete file path
    path = test_audio_dir + afile + '.ogg'
    
    # Open file with librosa and split signal into 5-second chunks
    # sig, rate = librosa.load(path)
    # ...
    
    # Let's assume we have a list of 12 audio chunks (1min / 5s == 12 segments)
    chunks = [[] for i in range(12)]

In [ ]:
chunks

In [ ]:
# to check our prediction, those are the strings we need to compare:
a = 0
for i in range(len(y_test)):
    if str(y_test[i])==str(predict_x[i]).replace(".", ""):
        a=a+1
print(a)
print(a/len(y_test))

In [ ]:
y_test.shape
predict_x.shape
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
#score = f1_score(str(y_test[1]),str(predict_x[1]).replace(".", ""), average = None)
#fil_acc_orig = accuracy_score(X_test,predict_x)

X_test

y



Let's prepare the test file to create the submision csv

In [ ]:
# we will divide the test audio in 12 audios of 5 seconds.

from pydub import AudioSegment
from pydub.utils import make_chunks 
sound = AudioSegment.from_file("../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg")

chunk_length_ms = 5000 # pydub calculates in millisec 
chunks = make_chunks(sound,chunk_length_ms) #Make chunks of one sec 
for i, chunk in enumerate(chunks): 
    chunk_name = "{0}.ogg".format(i) 
    print ("exporting", chunk_name) 
    chunk.export(chunk_name, format="ogg") 


In [ ]:
# Lets stract the features
import librosa
import numpy as np

def features_extractor(file):
    #load the file (audio)
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    #we extract mfcc
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    #in order to find out scaled feature we do mean of transpose of value
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features


import os
extracted_features_=[]
for i in range(0,12):
    file_name = os.path.join(os.path.abspath("./"),str(i) + ".ogg")
    data=features_extractor(file_name)
    extracted_features_.append([data])

In [ ]:
len(extracted_features_)

In [ ]:
import pandas as pd
extracted_features_df_=pd.DataFrame(extracted_features_,columns=['feature'])

In [ ]:
X_=np.array(extracted_features_df_['feature'].tolist())
X_.shape

In [ ]:
with open('../input/birdclef-2022/scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)

In [ ]:
scored_birds_ = to_categorical(labelencoder.fit_transform(scored_birds))

In [ ]:
scored_birds_

In [ ]:
len(scored_birds)

In [ ]:
score = []
scored_birds_.tolist()
predict_x.tolist()
for i in range(len(scored_birds_)):
    for j in range(len(predict_x)):
        if str(scored_birds_[i]).replace(".", "") == str(predict_x[j]).replace(".", ""):
            score.append(scored_birds[i])
print(score)


In [ ]:
predict__=model.predict(X_)

In [ ]:
len(predict__)

In [ ]:
# This is where we will store our results
pred = {'row_id': [], 'target': []}

# Process audio files and make predictions
for afile in file_list:
    
    # Complete file path
    path = test_audio_dir + afile + '.ogg'
    
    # Open file with librosa and split signal into 5-second chunks
    # sig, rate = librosa.load(path)
    # ...
    
    # Let's assume we have a list of 12 audio chunks (1min / 5s == 12 segments)
    chunks = [[] for i in range(12)]
    
    # Make prediction for each chunk
    # Each scored bird gets a random value in our case
    # since we don't actually have a model
    for i in range(len(chunks)):        
        chunk_end_time = (i + 1) * 5
        j=0
        for bird in scored_birds:
            
            # This is our random prediction score for this bird
            score = predict__[i][j]
            maximun = max(predict__[i])
            j=j+1
            # Assemble the row_id which we need to do for each scored bird
            row_id = afile + '_' + bird + '_' + str(chunk_end_time)
            
            # Put the result into our prediction dict and
            # apply a "confidence" threshold of 0.5
            pred['row_id'].append(row_id)
            pred['target'].append(True if score == maximun else False)

In [ ]:
pred

In [ ]:
# from sklearn.metrics import f1_score
#f1_score(y_true, y_pred, average=None)

In [ ]:
#Of course it is not a good way to solve the problem, we did our data analisys for something
# and the code will be of course improved. We need to use data from all the birds, consider
# the rating, etc
df_train.head()

In [ ]:
#Let's see how important are the audios with 0.o rating
count = 0
print(len(df_train))
for i in range(len(df_train)):
    if df_train['rating'][i] == 0.0:
        count = count +1
        print(df_train['primary_label'][i],df_train['time'][i])
print(count)

In [ ]:
#Let's check the duration of the unescored audios and see if those are very important factors
#in our analysis.
count = 0
print(len(df_scored))
for i in index_row_scored:
    if df_scored['rating'][i] == 0.0:
        count = count+1
        print(df_scored['primary_label'][i],df_scored['time'][i])
print(count)

In [ ]:
count = 0
print(len(df_scored))
for i in index_row_scored:
    print(df_scored['primary_label'][i],df_scored['time'][i])
    count = count+1

print(count)

In [ ]:
# The unrated audios doesn't look like very important data, those have average duration and
# there are not so many of them, and are not present in the birds with too less audios.
# We will drop them for now.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(random_state=0)
cls.fit(X_train, y_train)

z = cls.predict_proba(X_test)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:


predict_prob=model.predict(X_test)

predict_classes=np.argmax(predict_prob,axis=1)

predict_prob

clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))

In [ ]:
y_train_ = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]
y_train_ = np.asarray(y_train_)
y_train_.shape
X_train.shape
k=np.array(extracted_features_df['primary_label'].tolist())
k = labelencoder.fit_transform(k)

In [ ]:
clf.fit(X, k)

In [ ]:
print(clf.named_steps['linearsvc'].coef_)

In [ ]:
print(clf.named_steps['linearsvc'].intercept_)

In [ ]:
pre = clf.score(X,k)
pre

In [ ]:
prediction = clf.decision_function(X)

In [ ]:
prediction

In [ ]:
print(prediction[0])
max_ = []
for i in range(len(prediction)):
    print(prediction[i])

    

In [ ]:
results = pd.DataFrame(pred, columns = ['row_id', 'target'])

# Quick sanity check
print(results.head()) 
    
# Convert our results to csv
results.to_csv("submission.csv", index=False)